### 데이터 전처리

In [1]:
#카테고리 정보와 리뷰 데이터 결합
brand = pd.read_csv('./datas/item_dfs_final.csv', index_col=0)

In [2]:
brand.rename(columns = {'NAME' : 'item'}, inplace = True)
brand.rename(columns={'BRAND':'company'}, inplace=True)

In [3]:
review_ = pd.read_csv('./datas/review_data_2.csv', index_col=0)

In [4]:
review = pd.merge(brand, review_, how='inner')

In [5]:
review.dropna(inplace=True)

In [6]:
review = review.drop(['Unnamed: 0.1'], axis=1)

In [7]:
review.isnull().sum()

ID          0
RANK        0
company     0
item        0
VOLUME      0
PRICE       0
RATE        0
category    0
keywords    0
size        0
price       0
id          0
gender      0
age         0
skin        0
score       0
review      0
dtype: int64

In [8]:
# 나이 데이터에서 '세' 삭제
review['age'] = review['age'].map(lambda x: x.replace('세', '')).str.strip()
review['age'] = pd.to_numeric(review['age'])
review['age'].dropna(inplace=True)
review['age'] = review['age'].astype(int)

In [9]:
review = review[review['age'] < 80]

In [10]:
review['age'] = review['age'].astype(int)

In [11]:
review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201417 entries, 0 to 201665
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ID        201417 non-null  int64  
 1   RANK      201417 non-null  object 
 2   company   201417 non-null  object 
 3   item      201417 non-null  object 
 4   VOLUME    201417 non-null  object 
 5   PRICE     201417 non-null  int64  
 6   RATE      201417 non-null  float64
 7   category  201417 non-null  object 
 8   keywords  201417 non-null  object 
 9   size      201417 non-null  object 
 10  price     201417 non-null  object 
 11  id        201417 non-null  object 
 12  gender    201417 non-null  object 
 13  age       201417 non-null  int64  
 14  skin      201417 non-null  object 
 15  score     201417 non-null  float64
 16  review    201417 non-null  object 
dtypes: float64(2), int64(3), object(12)
memory usage: 27.7+ MB


In [12]:
# 리뷰 개수가 1개 이하인 id는 제외

In [13]:
review_count = review.groupby('id')['score'].count().reset_index().sort_values('score', ascending=False)
review_id = review_count['id'][review_count['score']>=2]

In [14]:
review = review[review['id'].isin(review_id)]
review.head()

,ID,RANK,company,item,VOLUME,PRICE,RATE,category,keywords,size,price,id,gender,age,skin,score,review
0,125591,1,더모테라피 (Dermo therapie),리페어레이션 에센스 토너,1000ml,7900,4.47,스킨/토너,"['수분공급', '유수분조절', '저자극', '피부진정']",1000ml,"7,900원",슬찡,여성,25,건성,3.0,양이 일단 너무 많고 진정팩으로 쓰기 딱인거 같아요!이거 양이 너무 많아서 사놓고 ...
1,125591,1,더모테라피 (Dermo therapie),리페어레이션 에센스 토너,1000ml,7900,4.47,스킨/토너,"['수분공급', '유수분조절', '저자극', '피부진정']",1000ml,"7,900원",다그닥다그다,여성,32,건성,5.0,스킨팩하기 넘 좋아요 미샤 5겹 화장솜에 적셔서 스킨팩하면 진정 쿨링👍
2,125591,1,더모테라피 (Dermo therapie),리페어레이션 에센스 토너,1000ml,7900,4.47,스킨/토너,"['수분공급', '유수분조절', '저자극', '피부진정']",1000ml,"7,900원",공듀다,여성,31,지성,4.0,양도 많고 토너 많이 쓰는데 이가격에 넘 좋음
3,125591,1,더모테라피 (Dermo therapie),리페어레이션 에센스 토너,1000ml,7900,4.47,스킨/토너,"['수분공급', '유수분조절', '저자극', '피부진정']",1000ml,"7,900원",소지닝겐,여성,34,복합성,4.0,이거는 진짜 가성비에 비해 대박임대용량 버전이 더 많이 생겼으면 좋겠어요화장솜 듬북...
4,125591,1,더모테라피 (Dermo therapie),리페어레이션 에센스 토너,1000ml,7900,4.47,스킨/토너,"['수분공급', '유수분조절', '저자극', '피부진정']",1000ml,"7,900원",kafukafu,여성,18,건성,4.0,양도 많구 흡수 잘되고 촉촉하고 가성비가 좋아서 학생들이 쓰기 참 좋습니다


### 4-2. 사용자 기반
#### (1) 원하는 검색 조건 입력

In [15]:
review['category'].unique()

array(['스킨/토너', '로션/에멀젼', '에센스/세럼', '크림', '미스트', '파운데이션', '피니시파우더', '컨실러',
       '베이스/프라이머', '립스틱', '립틴트/라커', '립펜슬', '립글로스', '립베이스', '립케어', '아이라이너',
       '아이브로우', '아이섀도우', '마스카라', '하이라이터', '블러셔', '쉐딩', '페이스오일', 'BB크림',
       'CC크림', '메이크업픽서', '아이프라이머', '컨투어링팔레트'], dtype=object)

In [16]:
category1 = input('화장품 카테고리를 입력하세요:')

화장품 카테고리를 입력하세요:로션/에멀젼


In [17]:
skin1 = input('피부 타입을 입력하세요 :')

피부 타입을 입력하세요 :건성


In [18]:
gender1 = input('성별을 입력하세요:')

성별을 입력하세요:여성


In [19]:
age1 = int(input('나이를 입력하세요:'))

나이를 입력하세요:18


In [20]:
review['skin']=review['skin'].str.strip()
review['category']=review['category'].str.strip()
review['gender']=review['gender'].str.strip()

In [21]:
filtered_id = review[(review['category']==category1 )& (review['skin']==skin1) & (review['gender']== gender1) & (review['age']==age1) & (review['score']==5)]

In [22]:
filtered_id = filtered_id[['item', 'company', 'price', 'RATE','review']]

In [23]:
filtered_id

,item,company,price,RATE,review
15835,세라마이드 하이드라 플루이드,라포티셀 (LAPOTHICELL),"36,000원",4.24,이거 같은라인 토너랑 같이 쓰는데 진짜 좋아요 !!촉촉해서 건성한테 잘맞아요 💓
16334,워터-풀 리밸런싱 젤 로션,숨37˚ (sum37˚),"53,000원",3.91,유분기도 없고 피부 트러블도 없고 산뜻하고 좋아요
16535,블루베리 리밸런싱 로션,이니스프리 (innisfree),"8,000원",3.81,일단 이 제품은 민감한 복합성에 여드름 피부인 제게 너무나 찰떡인 제품이에요건성에게...
16678,수분가득 콜라겐 에멀전,에뛰드 (ETUDE),"15,000원",3.73,여름뮤트🍎/건성/23-24호/원제품발림성이 되게 가벼워요. 흘러내릴 듯할 정도로 부...
18907,크리미 모이스춰라이저 딥 모이스트,빌리프 (belif),"40,000원",4.20,요즘 아침에는 얼굴이 너무 당기고 외출하고 집에오면 기름이 엄청뜨는데 그러면서 자연...
18964,에이리페어 오리지날 로션,셀라피 (Cellapy),"44,000원",4.16,엄펑 촉촉하고 무겁지도않아서 부드럽게발리고 보습할때 너무너무 좋아요!
19157,데일리 모이스쳐 테라피 페이셜 로션,피지오겔 (PHYSIOGEL),"31,900원",3.96,인생로션인 것 같아요 건조할 때 막 바르기도 좋고 양도 은근 많고 무엇보다 무취!!...
20093,촉촉 레서피 모이스처 로션,티엔 (tn),"8,200원",3.75,로션이랑 토너 둘 다 사용중인데 건성피부에 딱 맞는 제품입니다 !! 제 피부가 예민...
20293,어반 에코 하라케케 에멀젼,더샘 (the SAEM),"14,000원",3.60,제 피부가 더샘이랑 잘맞아서 써봤는데 너무 좋아요ㅠㅠ
20526,하이포알러제닉 모이스처라이징 에몰리언트,더마토리 (DERMATORY),"17,000원",4.08,진짜 이 에멀전?크림 제가 여태까지 써본 에멀전중에 제일 보습감 있어요! 저는 에멀...


### 4-2. 사용자 기반
#### (2) 전체 화장품에서 사용자 기반 협업 필터링을 통한 추천시스템

In [24]:
filtered_id = review

In [25]:
print(filtered_id.shape)

(181419, 17)


In [26]:
# 회사명, 제품명, 사용자id, 평점, 성별, 나이, 피부타입 가져오기
df = filtered_id[['id','item','score']]
print(df.shape)
df.head()

(181419, 3)


,id,item,score
0,슬찡,리페어레이션 에센스 토너,3.0
1,다그닥다그다,리페어레이션 에센스 토너,5.0
2,공듀다,리페어레이션 에센스 토너,4.0
3,소지닝겐,리페어레이션 에센스 토너,4.0
4,kafukafu,리페어레이션 에센스 토너,4.0


### 추천 시스템에 활용할 알고리즘 선택 위해, 알고리즘 별 성능 확인

In [27]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['id', 'item', 'score']], reader)

In [30]:
# Split data into 5 folds
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic

# SVD
algo = SVD()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print('\n')

# NMF
algo = NMF()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print('\n')

#KNNBasic
algo = KNNBasic()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9260  0.9235  0.9228  0.9243  0.9211  0.9235  0.0016  
Fit time          6.11    6.16    6.21    6.55    6.79    6.36    0.26    
Test time         0.27    0.31    0.29    0.33    0.33    0.30    0.02    


Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1074  1.1083  1.1152  1.1117  1.1083  1.1102  0.0029  
Fit time          10.15   9.94    10.34   10.15   10.05   10.13   0.13    
Test time         0.27    0.30    0.15    0.15    0.19    0.21    0.06    


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Comput

{'test_rmse': array([1.07999842, 1.07658866, 1.0808096 , 1.07767372, 1.07824478]),
 'fit_time': (26.455007791519165,
  30.295252799987793,
  30.60814094543457,
  32.87116813659668,
  31.157373905181885),
 'test_time': (2.0401721000671387,
  2.299168825149536,
  2.2241480350494385,
  2.309541940689087,
  1.915895938873291)}

### KNN Basic 활용하여 추천 시스템 구축

In [30]:
# 데이터 변형 (테이블 -> 딕셔너리)

def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:, 1:]) for k, g in grouped}
    return d

In [31]:
df_to_dict = recur_dictify(df)
df_to_dict

{'!!o-o': {'더 심플 데일리 로션': 4.0,
  '립 슬리핑 마스크': 4.0,
  '에이 클리어링 모이스처라이저': 4.0,
  '원더 세라마이드 모찌 토너': 1.0},
 '!BURU!': {'립 메덱스': 5.0,
  '마데카소사이드 시카 겔': 5.0,
  '마이 파운데이션 올데이롱웨어': 4.0,
  '메이크업 킵 베이스 [SPF28/PA++]': 1.0,
  '샤인픽스 아이즈 AD': 4.0,
  '샤인픽스 아이즈 글리터': 4.0,
  '스트레스 아웃 솔루션 패드 - 당근패드': 3.0,
  '연어 다크서클 컨실러 크림': 1.0,
  '후르츄 틴트': 3.0},
 '!_!': {'닥터솔루션 아큐어 리퀴드 각질 클리어링 솔루션': 4.0, '더마 플랜 센서티브 수딩 트리트먼트': 3.0},
 '!ㅎㅅㅎ': {'순정 약산성 5.5 진정 토너': 4.0, '크리스탈 미니 쿠션 블러셔': 4.0},
 '!김수진': {'에센스 커버 팩트 트리플 [SPF50+/PA+++]': 4.0, '타원형 오토 아이브로우': 5.0},
 '!띠드버거!': {'닥터솔루션 아큐어 클래리파잉 스팟 세럼': 3.0, '닥터솔루션 아큐어 클래리파잉 에멀전': 4.0},
 '!인스피릿!': {'AC 클리어 크림 플러스': 4.0,
  '드로잉 아이 브라우': 3.0,
  '미네랄 모노 섀도우 매트': 2.0,
  '샘물 우드 아이브로우': 4.0,
  '시카 히알루로닉 카밍앰플': 4.0,
  '약산성 18 퍼스트 토너': 3.0,
  '어린쑥 수분진정 크림': 2.0},
 '"뚥': {'더 퍼스트 젤 아이라이너': 4.0,
  '롱래스팅 팁 컨실러 빅 [SPF28/PA++]': 5.0,
  '립 슬리핑 마스크': 4.0,
  '무드 시네마 매트 엔딩': 5.0,
  '뮤제너 앰플 미스트': 3.0,
  '미니앨범 1집 [쉐딩]': 5.0,
  '바이플라워 컨투어링': 3.0,
  '사랑해주 톤업 크림': 2.0,
  '알래스카85 내추럴 토너': 4.0,
  '울트라

In [32]:
# 사용자 목록, 화장품 목록을 리스트로 담기

id_list = []
cos_set = set()

for user_key in df_to_dict:
    id_list.append(user_key)
    
    for cos_key in df_to_dict[user_key]:
        cos_set.add(cos_key)

In [33]:
cos_set

{'믹스컬러치크 (하이라이터)',
 '아이엠 매트 립스틱',
 '익스트림 립 리페어',
 '에코 소울 루미너스 리얼 광채 미스트',
 '플레이 101 펜슬 [립]',
 '모이스처라이징 립밤 클래식 튜브',
 '에이:쉐이프 오브 마이 페이스 트리블 컨투어',
 '토너워시',
 '마카롱 크림 필링 치크',
 '라이트 리플렉팅 루스 세팅 파우더',
 '쟈스민 3D 모이스트 팩트',
 '리퀴드 아이 쉬머',
 '클래식 필름 치크스',
 '세미 프로 멀티 컬러',
 'CC 쿨링 쿠션 [SPF42/PA+++]',
 '쉬머 아이라인 섀도우',
 '브로우래스팅 필오프 젤',
 '청귤 비타 C 토너 패드',
 '그레이프 워터',
 '히로인 스무스 커버 파우더',
 '립 메덱스',
 '잇츠 탑 프로페셔널 아이브로우 메이커',
 '자음수',
 '스테이지 퍼포머 인비저블 파우더 (슈:블러 파우더)',
 '시크릿 리페어 토너',
 '치크톤 싱글 블러셔',
 '쉬머링 페이셜 휩',
 '루즈 아르마니',
 '파워 10 포뮬라 엘아이 오일 앰플',
 '파워10 포뮬라 와이이 이펙터',
 '더마뉴얼 크림',
 '아이브로우 키트',
 '슈가 펀치 립 트리트먼트 [SPF15]',
 '더 스킨케어 파운데이션 [SPF23/PA++]',
 '글래머로즈 메이크업 픽서',
 '프린세스 앙투아네트 젤펜슬 아이라이너',
 '글라이딩 틴트',
 '에센셜 틴티드 립 글로우',
 '에어 핏 촉촉 틴트',
 '허브 워터 미스트 EX',
 '시크릿오일',
 '퍼펙트 커버 쿠션 [SPF50+/PA+++]',
 '원더포어 밸런싱 크림',
 '하이퍼 타이트 라이너',
 '솔루션 미 마일드 수딩 패드',
 '브로우 디자이너 팩트',
 '볼륨 픽스 마스카라',
 '살롱 드 카라 백 콤',
 '원데이 타투 틴트',
 '아이스드 타임 퍼밍 오일앰플',
 '샘물 광채 리퀴드 하이라이터',
 '셀라비브 리플레니싱 나이트 크림',
 '잉크 타투 스틱',
 '에센스 커버 팩트 VX [SPF50+/PA+++]',
 'BB 프레스 파우더

In [34]:
cos_list = list(cos_set)

In [35]:
# 추천 시스템에 사용할 rating 딕셔너리 만들기(닉네임, 제품이름, 평점)

In [36]:
rating_dic= {
    'id' : [],
    'item' : [],
    'score':[],
}

In [37]:
#사용자 수 만큼 반복
for name_key in df_to_dict:
    
    #해당 사용자가 본 화장품 수 만큼 반복
    for cos_key in df_to_dict[name_key]:
        
        #사용자 인덱스
        a1 = id_list.index(name_key)
        
        #화장품 인덱스
        a2 = cos_list.index(cos_key)
        
        #평점
        a3 = df_to_dict[name_key][cos_key]
        
        #딕셔너리에 추가
        rating_dic['id'].append(a1)
        rating_dic['item'].append(a2)
        rating_dic['score'].append(a3)

In [38]:
# 사용자 - 화장품 - 평점
print(len(rating_dic['id']))
print(len(rating_dic['item']))
print(len(rating_dic['score']))

181021
181021
181021


In [39]:
df = pd.DataFrame(rating_dic)
df.tail()

,id,item,score
181016,30046,4063,3
181017,30046,223,4
181018,30046,1112,5
181019,30046,2708,4
181020,30046,1875,3


In [40]:
# (2,)배열 형태로 데이터가 들어가있는 score가 있어서, 제거해줘야 함.
for i in range(len(df)):
    if df['score'][i].shape != ():
        df['score'][i] = df['score'][i][0]

In [41]:
df['score'] = pd.to_numeric(df['score'])

In [42]:
from surprise import Reader, Dataset
reader = Reader(rating_scale=(1, 5))
reader

In [43]:
col_list = ['id', 'item', 'score']
data = Dataset.load_from_df(df[col_list], reader)

In [44]:
# 학습
import surprise
trainset = data.build_full_trainset()
option = {"name" : "pearson"}
algo = surprise.KNNBasic(sim_options=option)
algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [45]:
# 사용자 id 기반 추천 시스템

id_recomm, cos_name, cos_comp, cos_price = list(), list(), list(), list()

user_id = input('ID를 입력하세요:  ')
print('\n')

idx = id_list.index(user_id)

knn_result = algo.get_neighbors(idx, k=5)
for i in knn_result:
    id_recomm.append(filtered_id['id'][filtered_id.index[i]])
    
print('유사한 사용자 ID는: {} 입니다.'.format(id_recomm))
print('\n')

print('추천 화장품 정보 : ')

for r1 in knn_result:
    max_rating = data.df[data.df['id']==r1]['score'].max()
    cos_id=data.df[(data.df['score']==max_rating)&(data.df['id']==r1)]['item'].values
    
    for cos_item in cos_id:
        cos_name.append(cos_list[cos_item]), 
        cos_comp.append(filtered_id['company'][filtered_id['item']==cos_list[cos_item]].unique()[0])
        cos_price.append(filtered_id['price'][filtered_id['item']==cos_list[cos_item]].unique()[0])
        
result = pd.DataFrame([cos_name, cos_comp, cos_price]).T
result.columns = ['제품명', '회사명', '가격']
result

ID를 입력하세요:  다연하트


유사한 사용자 ID는: ['Hyungenie', '슬찡', '다그닥다그다', '공듀다', '소지닝겐'] 입니다.


추천 화장품 정보 : 


,제품명,회사명,가격
0,생과일 립 앤 치크 세가지맛,스킨푸드 (SKINFOOD),"11,900원"
1,인리치드 립 에센스 그레이프,어라운드미 (AROUND ME),"3,500원"
2,킬 브로우 오토 하드 브로우 펜슬,클리오 (CLIO),"20,000원"
3,플레이 컬러 아이즈 [인 더 까페],에뛰드 (ETUDE),"22,000원"
4,더 심플 데일리 로션,싸이닉 (SCINIC),"18,000원"
5,립 슬리핑 마스크,라네즈 (LANEIGE),"18,000원"
6,에이 클리어링 모이스처라이저,클린앤클리어 (Clean&Clear),"12,900원"
7,립 메덱스,블리스텍스 (Blistex),"4,000원"
8,마데카소사이드 시카 겔,어퓨 (Apieu),"14,000원"
9,닥터솔루션 아큐어 리퀴드 각질 클리어링 솔루션,케어존 (CAREZONE),"28,000원"
